In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import os

from sklearn.preprocessing import StandardScaler
# from pygrinder import mcar
# from pypots.data import load_specific_dataset
from pypots.imputation import SAITS

In [3]:
path="/raid/abhilash/classification_datasets/"

In [53]:
datasets=['Epilepsy']#EMG']#,'Epilepsy', 'Gesture']
exc = ['SleepEEG', 'FD-B']

In [23]:
batches={
    'EMG':500}

def perform_saits_imputation(X, batch_size=None):
    
    imputed_samples = []
    
    original_seq_len = X.shape[1]
    if batch_size is not None:
        seq_len = batch_size
    else:
        seq_len = X.shape[1]
    
    num_samples = X.shape[0]
    num_feats = X.shape[-1]
    
    X = X.numpy()
    ss = StandardScaler()
    X = ss.fit_transform(X.reshape(-1, num_feats))
    X = X.reshape(-1, seq_len, num_feats)
    Xtrain = {"X": X}
    saits = SAITS(n_steps=seq_len, 
                  n_features=num_feats, 
                  n_layers=2, 
                  d_model=256, 
                  d_ffn=128, 
                  n_heads=4, 
                  d_k=64, 
                  d_v=64, 
                  dropout=0.1,
                  epochs=25,
                  device='cuda:2')
    '''
    TRAINING
    '''
    saits.fit(Xtrain)

    '''
    INFERENCE
    '''
    imputation = saits.impute(Xtrain)

    imputed_np = ss.inverse_transform(imputation.reshape(-1, num_feats))
    imputed_np = imputed_np.reshape(num_samples, original_seq_len, num_feats)
    return torch.tensor(imputed_np)

In [56]:
pvalues = ['p2', 'p4', 'p6', 'p8']
datasets=['SleepeeG']
for dataset in datasets:
    print(f"Dataset = {dataset} \n")
    batch_size = None#batches[dataset]
    for pvalue in tqdm(pvalues):
        
        print(f"p value = {pvalue}")
        
        for trial in range(0, 5):
            
            print(f"\nTRIAL = {trial}")
            
            train=torch.load(os.path.join(path,dataset,'train_' + pvalue + '_v' + str(trial) + '.pt'))
            trainX = train['samples'].transpose(1,2)

            val=torch.load(os.path.join(path,dataset,'val_' + pvalue + '_v' + str(trial) + '.pt'))
            valX = val['samples'].transpose(1,2)

            test=torch.load(os.path.join(path,dataset,'test_' + pvalue + '_v' + str(trial) + '.pt'))
            testX = test['samples'].transpose(1,2)

            '''
            train
            '''
            train_imputed = perform_saits_imputation(trainX, batch_size)
            torch.save({'samples':train_imputed.transpose(2,1), 'labels':train['labels']}, os.path.join(path,dataset,'train_'+pvalue+'_SAITS_imputed_' + 'v' + str(trial) + '.pt'))
            print(f"Completed train set")

            '''
            val
            '''
            val_imputed = perform_saits_imputation(valX, batch_size)
            torch.save({'samples':val_imputed.transpose(2,1), 'labels':val['labels']}, os.path.join(path,dataset,'val_'+pvalue+'_SAITS_imputed_' + 'v' + str(trial) + '.pt'))
            print(f"Completed val set")

            '''
            test
            '''
            test_imputed = perform_saits_imputation(testX, batch_size)
            torch.save({'samples':test_imputed.transpose(2,1), 'labels':test['labels']}, os.path.join(path,dataset,'test_'+pvalue+'_SAITS_imputed_' + 'v' + str(trial) + '.pt'))
            print(f"Completed test set")

Dataset = Epilepsy 



  0%|                                                                                                                       | 0/4 [00:00<?, ?it/s]2024-08-03 19:20:31 [INFO]: Using the given device: cuda:2
2024-08-03 19:20:31 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2024-08-03 19:20:31 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 1,318,584
2024-08-03 19:20:31 [INFO]: Epoch 001 - training loss: 2.2868
2024-08-03 19:20:31 [INFO]: Epoch 002 - training loss: 1.7695


p value = p2

TRIAL = 0


2024-08-03 19:20:32 [INFO]: Epoch 003 - training loss: 1.7264
2024-08-03 19:20:32 [INFO]: Epoch 004 - training loss: 1.3882
2024-08-03 19:20:32 [INFO]: Epoch 005 - training loss: 1.4025
2024-08-03 19:20:32 [INFO]: Epoch 006 - training loss: 1.2256
2024-08-03 19:20:32 [INFO]: Epoch 007 - training loss: 1.3062
2024-08-03 19:20:32 [INFO]: Epoch 008 - training loss: 1.1668
2024-08-03 19:20:32 [INFO]: Epoch 009 - training loss: 1.2107
2024-08-03 19:20:32 [INFO]: Epoch 010 - training loss: 1.1671
2024-08-03 19:20:32 [INFO]: Epoch 011 - training loss: 1.1856
2024-08-03 19:20:32 [INFO]: Epoch 012 - training loss: 1.1428
2024-08-03 19:20:32 [INFO]: Epoch 013 - training loss: 1.1769
2024-08-03 19:20:32 [INFO]: Epoch 014 - training loss: 1.1307
2024-08-03 19:20:32 [INFO]: Epoch 015 - training loss: 1.0947
2024-08-03 19:20:32 [INFO]: Epoch 016 - training loss: 1.0907
2024-08-03 19:20:32 [INFO]: Epoch 017 - training loss: 1.0879
2024-08-03 19:20:32 [INFO]: Epoch 018 - training loss: 1.0869
2024-08-

Completed train set


...

Completed train set


2024-08-03 20:36:00 [INFO]: Epoch 007 - training loss: 1.3907
2024-08-03 20:36:00 [INFO]: Epoch 008 - training loss: 1.5212
2024-08-03 20:36:00 [INFO]: Epoch 009 - training loss: 1.4735
2024-08-03 20:36:00 [INFO]: Epoch 010 - training loss: 1.2391
2024-08-03 20:36:00 [INFO]: Epoch 011 - training loss: 1.1879
2024-08-03 20:36:01 [INFO]: Epoch 012 - training loss: 1.4236
2024-08-03 20:36:01 [INFO]: Epoch 013 - training loss: 1.3938
2024-08-03 20:36:01 [INFO]: Epoch 014 - training loss: 1.2142
2024-08-03 20:36:01 [INFO]: Epoch 015 - training loss: 1.1886
2024-08-03 20:36:01 [INFO]: Epoch 016 - training loss: 1.1523
2024-08-03 20:36:01 [INFO]: Epoch 017 - training loss: 1.2981
2024-08-03 20:36:01 [INFO]: Epoch 018 - training loss: 1.2598
2024-08-03 20:36:01 [INFO]: Epoch 019 - training loss: 1.2020
2024-08-03 20:36:01 [INFO]: Epoch 020 - training loss: 1.1198
2024-08-03 20:36:01 [INFO]: Epoch 021 - training loss: 1.2012
2024-08-03 20:36:01 [INFO]: Epoch 022 - training loss: 1.2092
2024-08-

Completed val set


2024-08-03 20:36:10 [INFO]: Epoch 001 - training loss: 0.6108
2024-08-03 20:36:19 [INFO]: Epoch 002 - training loss: 0.4879
2024-08-03 20:36:28 [INFO]: Epoch 003 - training loss: 0.4486
2024-08-03 20:36:38 [INFO]: Epoch 004 - training loss: 0.4190
2024-08-03 20:36:47 [INFO]: Epoch 005 - training loss: 0.4026
2024-08-03 20:36:57 [INFO]: Epoch 006 - training loss: 0.3863
2024-08-03 20:37:06 [INFO]: Epoch 007 - training loss: 0.3789
2024-08-03 20:37:16 [INFO]: Epoch 008 - training loss: 0.3674
2024-08-03 20:37:25 [INFO]: Epoch 009 - training loss: 0.3558
2024-08-03 20:37:34 [INFO]: Epoch 010 - training loss: 0.3465
2024-08-03 20:37:44 [INFO]: Epoch 011 - training loss: 0.3455
2024-08-03 20:37:53 [INFO]: Epoch 012 - training loss: 0.3422
2024-08-03 20:38:02 [INFO]: Epoch 013 - training loss: 0.3339
2024-08-03 20:38:12 [INFO]: Epoch 014 - training loss: 0.3265
2024-08-03 20:38:21 [INFO]: Epoch 015 - training loss: 0.3268
2024-08-03 20:38:31 [INFO]: Epoch 016 - training loss: 0.3243
2024-08-

Completed test set


In [22]:
tr=torch.load(os.path.join(path, 'SleepEEG', 'train.pt'))
vl=torch.load(os.path.join(path, 'SleepEEG', 'val.pt'))
tst=torch.load(os.path.join(path, 'SleepEEG', 'test.pt'))
data=torch.cat((tr['samples'].transpose(1,2),vl['samples'].transpose(1,2),tst['samples'].transpose(1,2)))
data.shape

torch.Size([569100, 178, 1])

In [34]:
pvalues = ['p8', 'p6', 'p4', 'p2']
datasets=['SleepEEG']
batch_size = None
for pvalue in tqdm(pvalues):
        
    print(f"p value = {pvalue}")

    for trial in range(0, 5):

        print(f"\nTRIAL = {trial}")

        
        train=torch.load(os.path.join(path,dataset,'train_' + pvalue + '_v' + str(trial) + '.pt'))
        trainX = train['samples'].transpose(1,2)

        val=torch.load(os.path.join(path,dataset,'val_' + pvalue + '_v' + str(trial) + '.pt'))
        valX = val['samples'].transpose(1,2)

        test=torch.load(os.path.join(path,dataset,'test_' + pvalue + '_v' + str(trial) + '.pt'))
        testX = test['samples'].transpose(1,2)
        
        t0 = time.time()
        '''
        train
        '''
        train_imputed = perform_saits_imputation(trainX, batch_size)
        torch.save({'samples':train_imputed.transpose(2,1), 'labels':train['labels']}, os.path.join(path,dataset,'train_'+pvalue+'_SAITS_imputed_' + 'v' + str(trial) + '.pt'))
        print(f"Completed train set")

        '''
        val
        '''
        val_imputed = perform_saits_imputation(valX, batch_size)
        torch.save({'samples':val_imputed.transpose(2,1), 'labels':val['labels']}, os.path.join(path,dataset,'val_'+pvalue+'_SAITS_imputed_' + 'v' + str(trial) + '.pt'))
        print(f"Completed val set")

        '''
        test
        '''
        test_imputed = perform_saits_imputation(testX, batch_size)
        torch.save({'samples':test_imputed.transpose(2,1), 'labels':test['labels']}, os.path.join(path,dataset,'test_'+pvalue+'_SAITS_imputed_' + 'v' + str(trial) + '.pt'))
        print(f"Completed test set")
                
        t1=time.time()
        print(f"\nCost = {t1-t0}s\n")

  0%|                                                                                                                                                                                       | 0/4 [00:00<?, ?it/s]

p value = p8

TRIAL = 0


2024-08-12 01:28:04 [INFO]: Using the given device: cuda:2
2024-08-12 01:28:04 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2024-08-12 01:28:04 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 1,318,584
2024-08-12 01:33:37 [INFO]: Epoch 001 - training loss: 0.6475
2024-08-12 01:39:07 [INFO]: Epoch 002 - training loss: 0.6283
2024-08-12 01:44:37 [INFO]: Epoch 003 - training loss: 0.6021
2024-08-12 01:50:08 [INFO]: Epoch 004 - training loss: 0.5924
2024-08-12 01:55:33 [INFO]: Epoch 005 - training loss: 0.5885
2024-08-12 02:01:03 [INFO]: Epoch 006 - training loss: 0.5868
2024-08-12 02:06:33 [INFO]: Epoch 007 - training loss: 0.5860
2024-08-12 02:12:05 [INFO]: Epoch 008 - training loss: 0.5849
2024-08-12 02:17:35 [INFO]: Epoch 009 - training loss: 0.5852
2024-08-12 02:23:06 [INFO]: Epoch 010 - training loss: 0.5846
2024-08-12 02:28:37 [INFO]: Epoch 011 - training loss: 0.5841
2024-08-12 02:34:09 [INFO]

Completed train set


2024-08-12 03:46:26 [INFO]: Using the given device: cuda:2
2024-08-12 03:46:26 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2024-08-12 03:46:26 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 1,318,584
2024-08-12 03:48:02 [INFO]: Epoch 001 - training loss: 0.6051
2024-08-12 03:49:39 [INFO]: Epoch 002 - training loss: 0.5192
2024-08-12 03:51:15 [INFO]: Epoch 003 - training loss: 0.5086
2024-08-12 03:52:52 [INFO]: Epoch 004 - training loss: 0.4791
2024-08-12 03:54:28 [INFO]: Epoch 005 - training loss: 0.4869
2024-08-12 03:56:04 [INFO]: Epoch 006 - training loss: 0.4638
2024-08-12 03:57:41 [INFO]: Epoch 007 - training loss: 0.4648
2024-08-12 03:59:17 [INFO]: Epoch 008 - training loss: 0.6226
2024-08-12 04:00:54 [INFO]: Epoch 009 - training loss: 0.4548
2024-08-12 04:02:30 [INFO]: Epoch 010 - training loss: 0.4592
2024-08-12 04:04:07 [INFO]: Epoch 011 - training loss: 0.4486
2024-08-12 04:05:43 [INFO]

...

Completed val set


2024-08-12 11:32:51 [INFO]: Using the given device: cuda:2
2024-08-12 11:32:51 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2024-08-12 11:32:52 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 1,318,584
2024-08-12 11:34:12 [INFO]: Epoch 001 - training loss: 0.4403
2024-08-12 11:35:33 [INFO]: Epoch 002 - training loss: 0.3820
2024-08-12 11:36:54 [INFO]: Epoch 003 - training loss: 0.3716
2024-08-12 11:38:15 [INFO]: Epoch 004 - training loss: 0.3630
2024-08-12 11:39:36 [INFO]: Epoch 005 - training loss: 0.3565
2024-08-12 11:40:57 [INFO]: Epoch 006 - training loss: 0.3571
2024-08-12 11:42:18 [INFO]: Epoch 007 - training loss: 0.3527
2024-08-12 11:43:39 [INFO]: Epoch 008 - training loss: 0.3470
2024-08-12 11:45:00 [INFO]: Epoch 009 - training loss: 0.3421
2024-08-12 11:46:21 [INFO]: Epoch 010 - training loss: 0.3349
2024-08-12 11:47:42 [INFO]: Epoch 011 - training loss: 0.3333
2024-08-12 11:49:03 [INFO]

Completed test set

Cost = 12802.55673289299s


TRIAL = 3


2024-08-12 12:06:53 [INFO]: Using the given device: cuda:2
2024-08-12 12:06:53 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2024-08-12 12:06:53 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 1,318,584
2024-08-12 12:12:26 [INFO]: Epoch 001 - training loss: 0.4350
2024-08-12 12:17:58 [INFO]: Epoch 002 - training loss: 0.3920
2024-08-12 12:23:31 [INFO]: Epoch 003 - training loss: 0.4073
2024-08-12 12:29:03 [INFO]: Epoch 004 - training loss: 0.3799
2024-08-12 12:34:35 [INFO]: Epoch 005 - training loss: 0.3720
2024-08-12 12:39:40 [INFO]: Epoch 006 - training loss: 0.3698
2024-08-12 12:45:07 [INFO]: Epoch 007 - training loss: 0.3640
2024-08-12 12:50:39 [INFO]: Epoch 008 - training loss: 0.3623
2024-08-12 12:56:12 [INFO]: Epoch 009 - training loss: 0.3601
2024-08-12 13:01:45 [INFO]: Epoch 010 - training loss: 0.3585
2024-08-12 13:07:17 [INFO]: Epoch 011 - training loss: 0.3576
2024-08-12 13:12:49 [INFO]

KeyboardInterrupt: 

In [ ]:
saits_sleepeeg = [12738.73, 12783.72, 12802.55]
mean=statistics.mean(saits_sleepeeg)
std=statisitcs.stdev(saits_sleepeeg)

In [9]:
for dataset in datasets:
    train=torch.load(os.path.join(path,dataset,'train.pt'))
    val=torch.load(os.path.join(path,dataset,'val.pt'))
    test=torch.load(os.path.join(path,dataset,'test.pt'))
    
    print(f"For dataset = {dataset}, we have \ntrain shape = {train['samples'].shape}; val shape = {val['samples'].shape}; test shape = {test['samples'].shape}")
    
    print(f"Number of labels = {train['labels'].unique()}\n")

For dataset = EMG, we have 
train shape = torch.Size([122, 1, 1500]); val shape = torch.Size([41, 1, 1500]); test shape = torch.Size([41, 1, 1500])
Number of labels = tensor([0, 1, 2])



In [25]:
!ls /raid/abhilash/classification_datasets/SleepEEG/

test_p2.pt		   train_p2.pt		       val_p2.pt
test_p2_spline_imputed.pt  train_p2_spline_imputed.pt  val_p2_spline_imputed.pt
test_p4.pt		   train_p4.pt		       val_p4.pt
test_p4_spline_imputed.pt  train_p4_spline_imputed.pt  val_p4_spline_imputed.pt
test_p6.pt		   train_p6.pt		       val_p6.pt
test_p6_spline_imputed.pt  train_p6_spline_imputed.pt  val_p6_spline_imputed.pt
test_p8.pt		   train_p8.pt		       val_p8.pt
test_p8_spline_imputed.pt  train_p8_spline_imputed.pt  val_p8_spline_imputed.pt
test.pt			   train.pt		       val.pt


In [2]:
def mask_data_randomly_torch(X, probability):
    # Generate a random tensor of the same shape as X
    random_mask = torch.rand(X.shape)
    
    # Create a mask where true values represent the data points to be masked
    mask = random_mask < probability
    
    # Clone the original tensor to avoid modifying it directly
    X_masked = X.clone()
    
    # Apply NaN where the mask is True
    X_masked[mask] = torch.tensor(float('nan'))
    
    return X_masked

def perform_spline_imputation(X_masked):
    # Convert each sample's masked tensor to a DataFrame, impute, and collect them back
    imputed_samples = []
    for i in range(X_masked.shape[0]):
        # Extract the sample, convert to numpy, and then to DataFrame
        sample_df = pd.DataFrame(X_masked[i].numpy())

        # Impute missing values using mean imputation for each sample
        imputed_sample_df = sample_df.interpolate(method='spline', order=2, limit_direction='both')

        # Collect the imputed DataFrame
        imputed_samples.append(imputed_sample_df)
    
    df_imputed = pd.concat(imputed_samples, ignore_index=True)
    imputed_array = df_imputed.to_numpy()
    original_shape_array = imputed_array.reshape(X_masked.shape[0], X_masked.shape[1], X_masked.shape[2])
    X_imputed = torch.tensor(original_shape_array)
    return X_imputed

In [26]:
tr=torch.load(os.path.join(path, 'SleepEEG', 'train.pt'))

In [27]:
tr['samples'].shape

torch.Size([371055, 1, 178])

In [28]:
vl=torch.load(os.path.join(path, 'SleepEEG', 'val.pt'))
vl['samples'].shape

torch.Size([107730, 1, 178])

In [29]:
tst=torch.load(os.path.join(path, 'SleepEEG', 'test.pt'))
tst['samples'].shape

torch.Size([90315, 1, 178])

In [30]:
data=torch.cat((tr['samples'].transpose(1,2),vl['samples'].transpose(1,2),tst['samples'].transpose(1,2)))
data.shape

torch.Size([569100, 178, 1])

In [31]:
import time

In [32]:
pvalues = [0.8]
dataset="SleepEEG"
for pvalue in tqdm(pvalues):
    print(f"For p value = {pvalue} \n")
    
#     for dataset in ['SleepEEG']:
#         print(f"Dataset = {dataset}")
#         train=torch.load(os.path.join(path,dataset,'train.pt'))
#         trainX = train['samples'].transpose(1,2)
        
#         val=torch.load(os.path.join(path,dataset,'val.pt'))
#         valX = val['samples'].transpose(1,2)
        
#         test=torch.load(os.path.join(path,dataset,'test.pt'))
#         testX = test['samples'].transpose(1,2)
        
    for trial in range(0, 5):
        print(f"\nTRIAL = {trial}\n")
        '''
        train
        '''
        data_masked = mask_data_randomly_torch(data, pvalue)
        train_masked = data_masked[:371055, :, :]
        val_masked = data_masked[371055:371055+107730, :, :]
        test_masked = data_masked[-90315:, :, :]
        
        torch.save({'samples':train_masked.transpose(2,1), 'labels':tr['labels']}, os.path.join(path,dataset,'train_p'+str(pvalue)[-1] + '_v' + str(trial) + '.pt'))
        print(f"Train dataset masked")
        
        torch.save({'samples':val_masked.transpose(2,1), 'labels':vl['labels']}, os.path.join(path,dataset,'val_p'+str(pvalue)[-1]+'_v' + str(trial) + '.pt'))
        print(f"val dataset masked")
        
        torch.save({'samples':test_masked.transpose(2,1), 'labels':tst['labels']}, os.path.join(path,dataset,'test_p'+str(pvalue)[-1]+'_v' + str(trial) + '.pt'))
        print(f"test dataset masked")
        t0 = time.time()
        data_imputed = perform_spline_imputation(data_masked)
        train_imputed = data_imputed[:371055, :, :]
        val_imputed = data_imputed[371055:371055+107730, :, :]
        test_imputed = data_imputed[-90315:, :, :]
        
        torch.save({'samples':train_imputed.transpose(2,1), 'labels':tr['labels']}, os.path.join(path,dataset,'train_p'+str(pvalue)[-1]+'_spline_imputed_' +'_v' + str(trial) + '.pt'))
        torch.save({'samples':val_imputed.transpose(2,1), 'labels':vl['labels']}, os.path.join(path,dataset,'val_p'+str(pvalue)[-1]+'_spline_imputed_' + '_v' + str(trial) + '.pt'))
        torch.save({'samples':test_imputed.transpose(2,1), 'labels':tst['labels']}, os.path.join(path,dataset,'test_p'+str(pvalue)[-1]+'_spline_imputed_' + '_v' + str(trial) + '.pt'))        
        t1=time.time()
        print(f"\nCost = {t1-t0}s\n")

  0%|                                                                                                                                                                                       | 0/1 [00:00<?, ?it/s]

For p value = 0.8 


TRIAL = 0

Train dataset masked
val dataset masked
test dataset masked


/home/abhilash22/venv/lib/python3.9/site-packages/scipy/interpolate/_fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)



Cost = 447.11502051353455s


TRIAL = 1

Train dataset masked
val dataset masked
test dataset masked

Cost = 439.88587617874146s


TRIAL = 2

Train dataset masked
val dataset masked
test dataset masked

Cost = 442.6124782562256s


TRIAL = 3

Train dataset masked
val dataset masked
test dataset masked

Cost = 445.4619810581207s


TRIAL = 4

Train dataset masked
val dataset masked
test dataset masked


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [39:05<00:00, 2345.91s/it]


Cost = 450.07281947135925s



In [20]:
p8_time = [483.5, 625.58, 481.95, 489.13, 483.21]
mean=statistics.mean(p8_time)
std=statistics.stdev(p8_time)
print(f"mean = {mean}\nstd = {std}")

mean = 512.674
std = 63.17693115370517


In [64]:
def plot(dataset, our_values, simmtm_values):
    pvalues = [0.0, 0.2, 0.4, 0.6, 0.8]
    plt.plot(pvalues, our_values, linestyle='--', markerstyle='*', label = 'Our')
    plt.plot(pvalues, simmtm_values, linestyle='--', markerstyle='s', label = 'SimMTM')
    plt.title(f"F1 Performance : Dataset {dataset}")
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
EMG_F1_Our = [59.82312925170068]
EMG_F1_SimMTM = []

In [66]:
precision = 71.39 / 100
recall = 58.64 / 100

# Calculate F1 score
F1 = 2 * (precision * recall) / (precision + recall)
F1

0.6438990386833807